# Temporal derivatives

We'll compare and contrast methods from
[Tripp](http://compneuro.uwaterloo.ca/publications/tripp2010.html)
and [Voelker](https://github.com/ctn-waterloo/summerschool2015/tree/master/tutorials/temprep).

In [ ]:
# Common imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import nengo
import nengo.utils.numpy as npext

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

In [ ]:
def test_deriv(deriv_func, deriv_args, dims, ramp=True, t=1.0, dt=0.001):
    with nengo.Network() as net:
        if ramp:
            timesteps = int(t / dt)
            ramp = np.concatenate([np.linspace(0, 1, timesteps/2), np.linspace(1, 0, timesteps/2)])
            proc = lambda time: ramp[int(time / dt) % ramp.size]
        else:
            proc = nengo.processes.BrownNoise()
        inp = nengo.Node(size_in=dims)
        for i in range(dims):
            nengo.Connection(nengo.Node(output=proc), inp[i])
        ea = nengo.networks.EnsembleArray(n_neurons=40, n_ensembles=dims)
        nengo.Connection(inp, ea.input)
        deriv = deriv_func(dimensions=dims, **deriv_args)
        nengo.Connection(ea.output, deriv.input)
        in_probe = nengo.Probe(inp, synapse=None)
        out_probe = nengo.Probe(deriv.output, synapse=0.01)
    print("%d neurons" % (sum(e.n_neurons for e in net.all_ensembles)))
    sim = nengo.Simulator(net)
    sim.run(t)

    inp = sim.data[in_probe]
    plt.subplot(2, 1, 1)
    plt.plot(sim.trange(), inp)
    plt.ylabel("Input")
    plt.subplot(2, 1, 2)
    plt.plot(sim.trange(), sim.data[out_probe])

### Tripp 1: Feedforward, intermediate population

In [ ]:
from phd.networks.derivatives import TrippFF

test_deriv(TrippFF, {}, dims=2)

### Tripp 2: Feedforward, different synapses

In [ ]:
from phd.networks.derivatives import TrippInt

test_deriv(TrippInt, {}, dims=2)

### Tripp 3: Feedback approximating 2

In [ ]:
from phd.networks.derivatives import TrippFBInt

test_deriv(TrippFBInt, {}, dims=2)

### Tripp 4: Butterworth filter

In [ ]:
from phd.networks.derivatives import TrippButterworth

test_deriv(TrippButterworth, {}, dims=2)

### Voelker

In [ ]:
from phd.networks.derivatives import Voelker

test_deriv(Voelker, {'delay': 0.01}, dims=2)